In [1]:
from func import read_sam, read_fasta, read_fastq, write_fastq, fastq_to_fasta, quality, read_seq_from_file, convert_to_phred, write_seq_in_file_with_length, write_seq_in_file_with_length_and_name, muscle, muscle_with_gap1000, per_nucl_in_cal, percetn_of_nucl_in_cal, count_gaps, count_nucl_in_cal, consensus, reverse_complement, delete_gap, delete_insert, sort_fasta, start_probability, matrix_of_MDI, probability_of_trans

In [1]:
!simlord --generate-reference 0.6 10000 --save-reference random.fasta -n 20  myreads -fl 10000

Time for reading/generating the reference: 0:00:00.000643 h
Time for simulation of 20 reads: 0:00:00.079374 h.


In [2]:


random_fasta = read_seq_from_file('random.fasta')
list_rand_seq = []
list_rand_seq.append(reverse_complement(random_fasta[0]))
write_seq_in_file_with_length('random_compl.fasta', list_rand_seq, 0, 0)



In [1]:
!simlord --read-reference random_compl.fasta -n 20  myreads_compl -fl 10000

Time for reading/generating the reference: 0:00:00.000112 h
Time for simulation of 20 reads: 0:00:00.075620 h.


In [7]:
from tqdm import tqdm
import re

path_to_reads = 'myreads.sam'

if path_to_reads[-3:] == 'sam':
    sequence = read_sam('myreads.sam')
elif path_to_reads[-5:] == 'fastq':
    sequence = read_fastq('myreads.fastq')
elif path_to_reads[-5:] == 'fasta':
    sequence = read_fasta('myreads.fasta')
write_seq_in_file_with_length('myreads.fasta', sequence, 0, 0)

# подсчёт средней длины последовательности, чтобы избавиться от слишком коротких и слишком длинных последовательностей
alligment = read_seq_from_file('myreads.fasta')
avg_length = sum(len(word) for word in alligment) / len(alligment)
list_allig = []
list_need_seq = []
for j, i in enumerate(alligment):
    if len(i) >= avg_length / 1.6 or len(i) <= avg_length / 0.66:
        list_allig.append(i)
        list_need_seq.append(j)
write_seq_in_file_with_length('myreads.fasta', list_allig, 0, 0)

# Стартовые позиции последовательности, у нас всегда 0
list_startpos = []
for i in list_allig:
    list_startpos.append(0)

# Узнавание какая последовательность самая длинная и её индекс
sequence = read_seq_from_file('myreads.fasta')
maxi_index = max([len(word) for word in sequence])
index_max = 0
for j, i in enumerate(sequence):
    if len(i) == maxi_index:
        index_max = j

# Основная часть, в которой сначала проверяется подходит ли нам последовательность учитывая стартовую позицию
# Далее отрезается часть последовательности от indi до indi + 80, где indi это индекс, который увеличивается на половину длины выравнивания 
# Далее считается количество пропусков и нуклеотидов и заменяется в изначальных последовательностях куски нуклеотидов на выравненные 
list_already_use = []
list_lish_gap = []
indi = 0
count_nucl = 0
for start in tqdm(range(0, 20000, 40)):
    list_use_seq = []
    list_use_seq_index = []
    

    for j, seq in enumerate(sequence):
        if count_nucl >= list_startpos[j]:
            if j not in list_already_use:
                sequence[j] = indi * '-' + seq + (10000 - len(seq) - list_startpos[j]) * '-'
                list_already_use.append(j)
            gap_flag = False
            for i in seq[indi:indi + 80]:
                if i != '-':
                    gap_flag = True
                    continue
            if gap_flag:
                list_use_seq.append(sequence[j])
                list_use_seq_index.append(j)    

    write_seq_in_file_with_length_and_name(f'allig/reads_{indi}-{indi + 80}.fasta', list_use_seq, list_use_seq_index, indi, indi + 80)
    muscle(f'allig/reads_{indi}-{indi + 80}.fasta', f'allig/allig_{indi}-{indi + 80}.fasta')

    list_seq_with_insert = []
    sorted_seqs = sort_fasta(f'allig/allig_{indi}-{indi + 80}.fasta')
    for rec in sorted_seqs:
        list_seq_with_insert.append(str(rec.seq))
    write_seq_in_file_with_length_and_name(f'allig/allig_{indi}-{indi + 80}.fasta', list_seq_with_insert, list_use_seq_index,  0, 0)

    alligment = read_seq_from_file(f'allig/allig_{indi}-{indi + 80}.fasta')



    tmp = int(0.5 * len(alligment[0]))

    list_allig_index = []
    list_count_gap = []
    for j, seq in enumerate(alligment):
        tmp2 = seq[:tmp].count('-')
        list_count_gap.append(tmp2)
        list_allig_index.append(tmp - tmp2)
    
    for i in range(len(alligment)):
        alligment[i] = alligment[i][:tmp]

    for j, i in enumerate(list_use_seq_index):
        if i == index_max:
            count_nucl += int(list_allig_index[j] * 0.93)
        sequence[i] = sequence[i][:indi] + alligment[j] + sequence[i][list_allig_index[j] + indi:]
    indi += tmp

# Дополнение строк до одинаковой длины
list_length = []
for i in sequence:
    list_length.append(len(i))
maxi = max(list_length)
for j, i in enumerate(sequence):
    while len(sequence[j]) < maxi:
        sequence[j] = sequence[j] + '-'

write_seq_in_file_with_length('allig_seq.fasta', sequence, 0, 0)

# Постройка консенсуса 
list_consensus = []
alligment = read_seq_from_file('allig_seq.fasta')
count_nucl = per_nucl_in_cal(alligment)
count_gap = count_gaps (alligment)
alligment = delete_gap(alligment, count_gap, count_nucl, 50 / 100)
# Запись множественного выравнивания в файл
write_seq_in_file_with_length('multiple_alignment.fasta', alligment, 0, 0)
pi = start_probability (alligment)
a = matrix_of_MDI(alligment)
A = probability_of_trans(alligment, a)
count = count_nucl_in_cal(alligment)
b = percetn_of_nucl_in_cal(count)
consensuss = consensus(alligment, b)
# Запись консенсуса в файл
write_seq_in_file_with_length('consensus.fasta', [consensuss], 0, 0)
# Расчёт качества
consi = read_seq_from_file('consensus.fasta')
multiple_allig = read_seq_from_file('multiple_alignment.fasta')
list_seq = consi + multiple_allig
write_seq_in_file_with_length('consensus_and_multiple.fasta', list_seq, 0, 0)
list_prob_quality = quality(list_seq)
phred_string = convert_to_phred(list_prob_quality)
write_fastq('consensus.fastq', consi[0][:-100], phred_string)


100%|██████████| 500/500 [00:06<00:00, 76.94it/s] 


In [8]:
phred_string



'~y~~~~u~yy~~~~y~~y~uu~~u~p~~yuy~~~~~uyy~~yy~u~~~~y~~upuuy~p~~~~yyyyy~u~yyu~~u~~u~~~yy~~~~~~yg~~~~yyy~y~~~~~~y~~uuyu~uyyuu~~~y~~y~yuy~~u~~y~~~~uuy~~u~~~u~~pu~~y~y~y~y~p~y~~~~~y~y~y~byy~y~uy~yyu~yy~y~~~uu~y~y~~~~yy~yyy~~~pyyu~~~~~~~y~y~~~y~~~u~yyy~~~~~~yyyp~y~~~yp~yyy~uyy~y~~y~~~~u~~y~u~~yy~yy~uu~p~~~~~u~yy~y~~yu~~uuypuy~~~uuuyy~y~u~~~~yy~~~u~u~y~~yyy~u~~~~~y~y~~y~~pk~~~~~y~~~y~u~~~~uy~~~~yyy~k~u~yyp~y~~~y~~kp~~yu~yy~u~yu~~y~u~~~~y~~y~u~y~~p~~~u~~~y~y~~~uyu~~yy~~y~~yu~pu~y~~y~~~y~yy~~yyyy~u~~y~~~~yyu~~~~yyyy~yyy~yyy~~y~~~p~p~~~ypppyy~~~~~~~uy~y~~yy~~~y~~u~~~~~~u~~~~~y~y~py~~~up~~~uyyyu~~y~y~u~yyu~~~y~~~yy~yp~yy~~~u~y~~~yp~~uy~~u~~~uk~~y~u~~~~py~uyyyyk~yy~~~yy~u~y~~k~~yyuyuyuy~~~~p~~~pyy~~~~y~u~~u~y~~y~~uyuyyyyy~y~~y~y~u~~yyy~~~~~u~~~upyu~~~u~~~y~~~~y~y~~~~~~yu~~~uyy~~y~~y~~~y~~~~~~u~~~~~u~p~y~~y~~gp~p~p~~y~p~y~~~~yyp~~~~~~y~y~~~yu~p~~~yypu~py~uyyp~~~~uuy~~y~~~y~~~~p~~yyy~~kyy~y~uy~uy~~y~u~~~~~y~~~~~~~~~~~~yy~y~u~~~y~~u~y~y~yu~~y~~~yuu~~u~yu~~y~~~y~~~yy~~yp~y~uy~p~yu~~~u~~u~y~~~~~y~~~~~~

In [ ]:
# Постройка консенсусов с разным процентом на пропуски, далее из них строится окончательный консенсус
list_consensus = []
for j in range(45, 76, 5):
    alligment = read_seq_from_file('allig_seq.fasta')
    count_nucl = per_nucl_in_cal(alligment)
    count_gap = count_gaps (alligment)
    alligment = delete_gap(alligment, count_gap, count_nucl, j / 100)
    if j == 50:
        write_seq_in_file_with_length('multiple_alignment.fasta', alligment, 0, 0)
    pi = start_probability (alligment)
    a = matrix_of_MDI(alligment)
    A = probability_of_trans(alligment, a)
    count = count_nucl_in_cal(alligment)
    b = percetn_of_nucl_in_cal(count)
    consensuss = consensus(alligment, b)
    if j == 50:
        write_seq_in_file_with_length('consensus_50.fasta', [consensuss], 0, 0)
    list_consensus.append(consensuss)
write_seq_in_file_with_length('list_cons.fasta', list_consensus, 0, 0)
muscle('list_cons.fasta', 'allig_cons.fasta')

list_seq_for_startpos = []
sorted_seqs = sort_fasta("allig_cons.fasta")
for rec in sorted_seqs:
    list_seq_for_startpos.append(str(rec.seq))
write_seq_in_file_with_length('allig_cons.fasta', list_seq_for_startpos, 0, 0)

alligment = read_seq_from_file('allig_cons.fasta')
count_nucl = per_nucl_in_cal(alligment)
count_gap = count_gaps (alligment)
alligment = delete_gap(alligment, count_gap, count_nucl, 50 / 100)
pi = start_probability (alligment)
a = matrix_of_MDI(alligment)
A = probability_of_trans(alligment, a)
count = count_nucl_in_cal(alligment)
b = percetn_of_nucl_in_cal(count)
consensuss = consensus(alligment, b)

write_seq_in_file_with_length('consensus.fasta', [consensuss], 0, 0)

Комплементарная

In [ ]:
from tqdm import tqdm
import re

sequence = []

with open("myreads_compl.sam", "r") as sam_file:
    for line in sam_file:
        if line.startswith("@"):  # пропускаем заголовочные строки
            continue
        parts = line.strip().split("\t")
        sequence.append(parts[9])  # получаем последовательность

write_seq_in_file_with_length('myreads_compl.fasta', sequence, 0, 0)

alligment = read_seq_from_file('myreads_compl.fasta')
avg_length = sum(len(word) for word in alligment) / len(alligment)


list_allig = []
list_need_seq = []
for j, i in enumerate(alligment):
    if len(i) >= avg_length / 1.6:
        list_allig.append(i)
        list_need_seq.append(j)

write_seq_in_file_with_length('myreads_compl.fasta', list_allig, 0, 0)



file_path = "myreads_compl.sam"  # указать путь к файлу
list_startpos = []

count_seq = 0
with open(file_path, "r") as file:
    for line in file:
        match = re.search(r"startpos=(\d+)", line)  # ищем подстроку "startpos=" и значения после нее
        if match:
            value = int(match.group(1))  # извлекаем значение и преобразуем его в целое число
            if 0 <= value <= 10000 and count_seq in list_need_seq:  # проверяем ограничение диапазона
                list_startpos.append(value)
                count_seq += 1
            else:
                count_seq += 1



indi = 0
count_nucl = 0
sequence = read_seq_from_file('myreads_compl.fasta')

maxi_index = max([len(word) for word in sequence])
index_max = 0
for j, i in enumerate(sequence):
    if len(i) == maxi_index:
        index_max = j

for start in tqdm(range(0, 30000, 40)):
    list_use_seq = []
    list_use_seq_index = []

    for j, seq in enumerate(sequence):
        if count_nucl + 20 > list_startpos[j] > count_nucl - 20:
            if list_startpos[j] > count_nucl:
                sequence[j] = indi * '-' + ('-' * (list_startpos[j] - count_nucl))  + seq + (10000 - len(seq) - list_startpos[j]) * '-'
                list_use_seq.append(indi * '-' + ('-' * (list_startpos[j] - count_nucl)) + seq)
            elif list_startpos[j] <= count_nucl:
                sequence[j] = indi * '-' + seq[count_nucl - list_startpos[j]:] + (10000 - len(seq) - list_startpos[j]) * '-'
                list_use_seq.append(indi * '-' + seq[count_nucl - list_startpos[j]:])
            list_use_seq_index.append(j)
        elif count_nucl > list_startpos[j]:
            gap_flag = False
            for i in seq[indi:indi + 80]:
                if i != '-':
                    gap_flag = True
                    continue
            if gap_flag:
                list_use_seq.append(seq)
                list_use_seq_index.append(j)
    print(count_nucl)
    write_seq_in_file_with_length_and_name('output.fasta', list_use_seq, list_use_seq_index, indi, indi + 80)
    muscle('output.fasta', 'allig.fasta')

    list_seq_with_insert = []
    sorted_seqs = sort_fasta("allig.fasta")
    for rec in sorted_seqs:
        list_seq_with_insert.append(str(rec.seq))
    write_seq_in_file_with_length_and_name('allig.fasta', list_seq_with_insert, list_use_seq_index,  0, 0)

    alligment = read_seq_from_file('allig.fasta')

    tmp = int(0.65 * len(alligment[0]))

    list_allig_index = []
    list_count_gap = []
    for j, seq in enumerate(alligment):
        tmp2 = seq[:tmp].count('-')
        list_count_gap.append(tmp2)
        
        list_allig_index.append(tmp - tmp2)
    
    for i in range(len(alligment)):
        alligment[i] = alligment[i][:tmp]
    

    for j, i in enumerate(list_use_seq_index):
        if i == index_max:
            count_nucl += int(list_allig_index[j] * 0.95)
        sequence[i] = sequence[i][:indi] + alligment[j] + sequence[i][list_allig_index[j] + indi:]
    indi += tmp
    

list_length = []

for i in sequence:
    list_length.append(len(i))

maxi = max(list_length)

for j, i in enumerate(sequence):
    while len(sequence[j]) < maxi:
        sequence[j] = sequence[j] + '-'

write_seq_in_file_with_length('allig_seq_compl.fasta', sequence, 0, 0)

list_consensus = []

for j in range(45, 86, 5):
    alligment = read_seq_from_file('allig_seq_compl.fasta')
    count_nucl = per_nucl_in_cal(alligment)
    count_gap = count_gaps (alligment)
    alligment = delete_gap(alligment, count_gap, count_nucl, j / 100)
    pi = start_probability (alligment)
    a = matrix_of_MDI(alligment)
    A = probability_of_trans(alligment, a)
    count = count_nucl_in_cal(alligment)
    b = percetn_of_nucl_in_cal(count)
    consensuss = consensus(alligment, b)
    list_consensus.append(consensuss)
write_seq_in_file_with_length('list_cons_compl.fasta', list_consensus, 0, 0)
muscle('list_cons_compl.fasta', 'allig_cons_compl.fasta')

list_seq_for_startpos = []
sorted_seqs = sort_fasta("allig_cons_compl.fasta")
for rec in sorted_seqs:
    list_seq_for_startpos.append(str(rec.seq))
write_seq_in_file_with_length('allig_cons_compl.fasta', list_seq_for_startpos, 0, 0)

list_consensus = []

for j in range(45, 86, 5):
    alligment = read_seq_from_file('allig_cons_compl.fasta')
    count_nucl = per_nucl_in_cal(alligment)
    count_gap = count_gaps (alligment)
    alligment = delete_gap(alligment, count_gap, count_nucl, j / 100)
    pi = start_probability (alligment)
    a = matrix_of_MDI(alligment)
    A = probability_of_trans(alligment, a)
    count = count_nucl_in_cal(alligment)
    b = percetn_of_nucl_in_cal(count)
    consensuss = consensus(alligment, b)
    list_consensus.append(consensuss)

write_seq_in_file_with_length('list_cons_from_cons_compl.fasta', list_consensus, 0, 0)

  0%|          | 0/750 [00:00<?, ?it/s]

0


  0%|          | 1/750 [00:00<03:47,  3.29it/s]

[0, 5, 7, 8, 11, 12, 13, 14]
37


  0%|          | 2/750 [00:00<03:52,  3.21it/s]

...

[12]
9379


100%|█████████▉| 749/750 [02:47<00:00,  5.44it/s]

[12]
9379


100%|██████████| 750/750 [02:47<00:00,  4.47it/s]

[12]


In [21]:
consi_compl = read_seq_from_file('list_cons_from_cons_compl.fasta')
right_consi_compl = consi_compl[1]
right_consi_compl = reverse_complement(right_consi_compl)

consi = read_seq_from_file('list_cons_from_cons.fasta')
right_consi = consi[1]

list_consi = [right_consi, right_consi_compl]
write_seq_in_file_with_length('consi_two_seq.fasta', list_consi, 0, 0)
muscle('consi_two_seq.fasta', 'consi_two_seq_allig.fasta')

two_seq = read_seq_from_file('consi_two_seq_allig.fasta')

count_nucl = per_nucl_in_cal(two_seq)
count_gap = count_gaps (two_seq)
two_seq = delete_gap(two_seq, count_gap, count_nucl, 70 / 100)
pi = start_probability (two_seq)
a = matrix_of_MDI(two_seq)
A = probability_of_trans(two_seq, a)
count = count_nucl_in_cal(two_seq)
b = percetn_of_nucl_in_cal(count)
consensuss = consensus(two_seq, b)

list_consi = []
list_consi.append(consensuss)
tmp = read_seq_from_file('random.fasta')
list_consi.append(tmp[0])

write_seq_in_file_with_length('consi.fasta', list_consi, 0, 0)

muscle('consi.fasta', 'consi_allig.fasta')

alligment = read_seq_from_file('consi_allig.fasta')

count_sovp = 0
for i in range(len(min(alligment))):
    if alligment[0][i] == alligment[1][i]:
        count_sovp += 1

print(len(alligment[0]))
print(count_sovp)

12653
6568


HMM

In [22]:
!hmmbuild allig_seq_hmmer allig_seq.fasta
!hmmbuild allig_seq_hmmer_compl allig_seq_compl.fasta
!hmmemit -c allig_seq_hmmer > allig_seq_hmmer.fasta
!hmmemit -c allig_seq_hmmer_compl > allig_seq_hmmer_compl.fasta

sequ_compl = read_seq_from_file('allig_seq_hmmer_compl.fasta')
sequ_compl[0] = reverse_complement(sequ_compl[0])
sequ = read_seq_from_file('allig_seq_hmmer.fasta')
sequ.append(sequ_compl[0])

write_seq_in_file_with_length('consi_two_seq_hmmer.fasta', sequ, 0, 0)
muscle('consi_two_seq_hmmer.fasta', 'consi_two_seq_allig_hmmer.fasta')

two_seq = read_seq_from_file('consi_two_seq_allig_hmmer.fasta')

count_nucl = per_nucl_in_cal(two_seq)
count_gap = count_gaps (two_seq)
two_seq = delete_gap(two_seq, count_gap, count_nucl, 60 / 100)
pi = start_probability (two_seq)
a = matrix_of_MDI(two_seq)
A = probability_of_trans(two_seq, a)
count = count_nucl_in_cal(two_seq)
b = percetn_of_nucl_in_cal(count)
consensuss = consensus(two_seq, b)


list_consi = []
list_consi.append(consensuss)
tmp = read_seq_from_file('random.fasta')
list_consi.append(tmp[0])

write_seq_in_file_with_length('consi.fasta', list_consi, 0, 0)

muscle('consi.fasta', 'consi_allig.fasta')

alligment = read_seq_from_file('consi_allig.fasta')

count_sovp = 0
for i in range(len(min(alligment))):
    if alligment[0][i] == alligment[1][i]:
        count_sovp += 1

print(len(alligment[0]))
print(count_sovp)

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             allig_seq.fasta
# output HMM file:                  allig_seq_hmmer
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen     W eff_nseq re/pos description
#---- -------------------- ----- ----- ----- ----- -------- ------ -----------
1     allig_seq               16 16037  9471 10435     1.86  0.621 

# CPU time: 0.95u 0.01s 00:00:00.96 Elapsed: 00:00:00.95
# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - -

выравнивание на драфт и последующая сборка чистового

In [ ]:
consi = read_seq_from_file('list_cons_from_cons.fasta')
right_consi = consi[5]

list_draft_and_reads = []
list_draft_and_reads.append(right_consi)
list_reads = read_seq_from_file('myreads.fasta')

for i in list_reads:
    list_draft_and_reads.append(i)

write_seq_in_file_with_length('draft_and_reads.fasta', list_draft_and_reads, 0, 0)
muscle('draft_and_reads.fasta', 'draft_and_reads_align.fasta')

In [ ]:
list_consensus = []

for j in range(45, 86, 5):
    alligment = read_seq_from_file('draft_and_reads_align.fasta')
    count_nucl = per_nucl_in_cal(alligment)
    count_gap = count_gaps (alligment)
    alligment = delete_gap(alligment, count_gap, count_nucl, j / 100)
    pi = start_probability (alligment)
    a = matrix_of_MDI(alligment)
    A = probability_of_trans(alligment, a)
    count = count_nucl_in_cal(alligment)
    b = percetn_of_nucl_in_cal(count)
    consensuss = consensus(alligment, b)
    list_consensus.append(consensuss)

In [ ]:
write_seq_in_file_with_length('hmm_draft_and_reads_align.fasta', list_consensus, 0, 0)

In [46]:
list_startpos

[432,
 1586,
 3716,
 471,
 0,
 1626,
 0,
 652,
 2888,
 798,
 538,
 0,
 895,
 3158,
 1167,
 1072]

In [47]:
triple_seq = read_seq_from_file('simlord_with_cigar.fasta')

In [48]:
for i in range(len(triple_seq)):
    triple_seq[i] = list_startpos[i] * '-' + triple_seq[i]
triple_seq

['------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------C-TATAGTTCACCG-CGCAAGGCGCAGTGCC-CGTGA-TCGCGTTGCGCCGGCC-GCCTCGGCAGTGACTCATCCGGTTGG-CGCGAGCCC-AGCGTAGCGCACGGCGATG-ACGTTCCCGCGAGTACTCTCC-TGAGGACGCGCACATC-CGCTGGGGCCACCTA-TCCCTAAC-GCGCGCAATGTAGCATCCGTACGATGTGTTTCTGACCACCCTCGCGGGGGCTCCCGTCCTTTCCCGATCAGTGCGTCACTATTCG--TAGACATGAGCCCGG-GCCCGCTTCTCCCCATCGG-CCTAGCCCCTCTCGGAACCCGCCATGAGCTAAAGGCCG-T--GCGGACCACAGCGGCAG-AGCTACGA-GCGCCGGAGCCACACCGTCCCA-CCCGGTCGCG-C-GGATGCAGCACCCATTATCT-TCA-GCAGTGGTTAGGATACCCACTATGTACT-AAAGTCCGCCCGGGAC-GTA-GGGGTAGGTGCGCTAAAG--ACAATCCTGTTGCGGGGACCT-CAGA-AGTGCTTGCTGGAATGTGGTGTAGCTTGT-AA-GGGGAAC

In [49]:
write_seq_in_file_with_length('simlord_with_cigar_right.fasta', triple_seq, 0, 0)

Сравнить с ХММ драфтовые консенсусы 
Выровнять на драфт
полировать с помощью arrow
Сделать свою полировку

In [ ]:

list_rand_seq = []
list_rand_seq.append(reverse_complement(consensuss))


In [ ]:
list_rand_seq

In [ ]:
write_seq_in_file_with_length('random_compl.fasta', list_rand_seq, 0, 0)

In [ ]:
muscle('consi.fasta', 'cons.fasta')

In [41]:
import re

sequence = []
cigar = []

with open("myreads.sam", "r") as sam_file:
    for line in sam_file:
        if line.startswith("@"):  # пропускаем заголовочные строки
            continue
        parts = line.strip().split("\t")
        sequence.append(parts[9])  # получаем последовательность
        cigar.append(parts[5])  # получаем строку CIGAR
        # далее можно использовать код, приведенный в предыдущем ответе, для обработки строки CIGAR и получения результирующей последовательности

In [42]:
print(*sequence, sep='\n')

...


In [43]:
print(cigar[0][0:100])
print(sequence[0][0:100])

1=1D8=1I4=1I1D15=1I1=1I1D5=1D1=1I2=1I7=1I6=1D15=1X9=1I1=1D3=1I6=1D19=1D12=1I8=1I1=1D11=1X4=1D1I10=1X
CTATAGTTCAACCGCCGCAAGGCGCAGTGCGCGCGTGATGCGTCGTTGCGACCGGCCGCCTCGGCAGTGACTCATCCGGTTGAGCGCGGAGCCCAGCGTA


In [44]:
import re

list_sequence_with_cigar = []

for i in range(len(sequence)):
    output_sequence = ""
    index = 0
# обрабатываем каждую команду в строке CIGAR
    for operation in re.findall("\d+[A-Z=]", cigar[i]):
        length = int(operation[:-1])  # длина операции
        action = operation[-1]  # тип операции
        if action in ["M", "X", "="]:  # если операция – матч или замена
            output_sequence += sequence[i][index:index+length]  # добавляем соответствующий участок последовательности
            index += length
        elif action == "D":  # если операция – удаление
            output_sequence += "-" * length  # добавляем пропуск заданной длины
        elif action == "I":  # если операция – вставка
            index += length  # пропускаем заданную вставку
        else:
            raise ValueError("Некорректный тип операции CIGAR")  # выбрасываем ошибку, если встретилась некорректная операция
    list_sequence_with_cigar.append(output_sequence)
print(*list_sequence_with_cigar, sep='\n')  # выводим получившуюся последовательность

C-TATAGTTCACCG-CGCAAGGCGCAGTGCC-CGTGA-TCGCGTTGCGCCGGCC-GCCTCGGCAGTGACTCATCCGGTTGG-CGCGAGCCC-AGCGTAGCGCACGGCGATG-ACGTTCCCGCGAGTACTCTCC-TGAGGACGCGCACATC-CGCTGGGGCCACCTA-TCCCTAAC-GCGCGCAATGTAGCATCCGTACGATGTGTTTCTGACCACCCTCGCGGGGGCTCCCGTCCTTTCCCGATCAGTGCGTCACTATTCG--TAGACATGAGCCCGG-GCCCGCTTCTCCCCATCGG-CCTAGCCCCTCTCGGAACCCGCCATGAGCTAAAGGCCG-T--GCGGACCACAGCGGCAG-AGCTACGA-GCGCCGGAGCCACACCGTCCCA-CCCGGTCGCG-C-GGATGCAGCACCCATTATCT-TCA-GCAGTGGTTAGGATACCCACTATGTACT-AAAGTCCGCCCGGGAC-GTA-GGGGTAGGTGCGCTAAAG--ACAATCCTGTTGCGGGGACCT-CAGA-AGTGCTTGCTGGAATGTGGTGTAGCTTGT-AA-GGGGAACGCCCCTGGGACCCTATCCGGACGTGCGACCTTCCGGTGTGACTTACTCCGATCTCAGAAGCCGCC-GG-ACGCGCGCCTGGTCAGACC-CACTACTGGC-CCGTTGGGGGGCCTAAGCAGAGGCAGCGTGAAGCTCCTCATGGCATTCGACG-G-TGTGAGCGGAGAGTGCCTGCCGCGATGCTAGAGCTCTC-CGGAGGTAAAGGCCGCACACGGTAGGTGG-CCGGTGTTCACACA-CCGATGTGAGGGGGGCTGCGTACGTACCCCGACGCATAGACGCAAACAATGTGCCCATTCACAGTCCAATAGGC-CACC-CCGGGGTGCCATGTGGCTGA-ACTGAGCAGG-TCCTCCTAGCACCCTCCCGGAGGATT-CAC-GTGCGCCAGATAGCGTTCCCGGCAGGGCTGGGCCGGTCGCCAGGAGCGGTCA

In [45]:
write_seq_in_file_with_length('simlord_with_cigar.fasta', list_sequence_with_cigar, 0, 0)

In [2]:
a = [78, -32, 5, 39, 58, -5, -63, 57, 72, 9, 53, -1, 63, -97, -21, -94, -47, 57, -8, 60, -23, -72, -22, -79, 90, 96, -41, -71, -48, 84, 89, -96, 41, -16, 94, -60, -64, -39, 60, -14, -62, -19, -3, 32, 98, 14, 43, 3, -56, 71, -71, -67, 80, 27, 92, 92, -64, 0, -77, 2, -26, 41, 3, -31, 48, 39, 20, -30, 35, 32, -58, 2, 63, 64, 66, 62, 82, -62, 9, -52, 35, -61, 87, 78, 93, -42, 87, -72, -10, -36, 61, -16, 59, 59, 22, -24, -67, 76, -94, 59]

n = len(a)

# реализация алгоритма сортировки выбором
for i in range(n - 1):
    maxi = -100000
    for j in range(n - i):
        if maxi < a[j]:
            maxi = a[j]
            index = j
    a[j], a[index] = a[index], a[j]
print(a)

[-97, -96, -94, -94, -79, -77, -72, -72, -71, -71, -67, -67, -64, -64, -63, -62, -62, -61, -60, -58, -56, -52, -48, -47, -42, -41, -39, -36, -32, -31, -30, -26, -24, -23, -22, -21, -19, -16, -16, -14, -10, -8, -5, -3, -1, 0, 2, 2, 3, 3, 5, 9, 9, 14, 20, 22, 27, 32, 32, 35, 35, 39, 39, 41, 41, 43, 48, 53, 57, 57, 58, 59, 59, 59, 60, 60, 61, 62, 63, 63, 64, 66, 71, 72, 76, 78, 78, 80, 82, 84, 87, 87, 89, 90, 92, 92, 93, 94, 96, 98]


In [5]:
list1 = [1,2,3,4]
list2 = [1,5,3,4, 5, 6]
print(list2.sort())
print(list1 + list2)
print((list1 + list2).sort())

None
[1, 2, 3, 4, 1, 3, 4, 5, 5, 6]
None
